In [ ]:
# Step-1: Model building
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 100
EPOCHS = 10

# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/TrimDataset/Dataset1/Train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/TrimDataset/Dataset1/Test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary')

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

# Save the model
model.save('/content/drive/MyDrive/cnn_image_classifier_model.h5')

Found 4235 images belonging to 2 classes.
Found 5337 images belonging to 2 classes.


In [ ]:
# Step-2: Using our model to classify images
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from google.colab import files

# Image size for resizing
IMAGE_SIZE = (128, 128, 3)

def preprocess_image(image, target_size):
    img = Image.open(image)
    img = img.resize(target_size[:2])
    img = np.array(img) / 255.0
    return img

model_path = '/content/drive/MyDrive/cnn_image_classifier_model.h5'
cnn_model = load_model(model_path)

while True:
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]
    preprocessed_image = preprocess_image(image_path, IMAGE_SIZE)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Add batch dimension
    prediction = cnn_model.predict(preprocessed_image)
    predicted_class_index = np.argmax(prediction)
    if predicted_class_index == 1:
        print("The image is classified as FAKE.")
    else:
        print("The image is classified as REAL.")
    choice = input("Do you want to upload and classify another image? (yes/no): ")
    if choice.lower() != 'yes':
        break


Saving real_1.jpg to real_1.jpg
1/1 [==============================] - 0s 173ms/step
The image is classified as REAL.
Do you want to upload and classify another image? (yes/no): yes


Saving fake_209.jpg to fake_209.jpg
1/1 [==============================] - 0s 31ms/step
The image is classified as REAL.
Do you want to upload and classify another image? (yes/no): no
